# Same fields as RubinTV has now for visitSummary table with multiple detectors


Uses np.mean to average all the detectors. I could envision a median, scatter, max, min etc... too

In [1]:
from lsst.daf.butler import Butler
import numpy as np

In [2]:
butler = Butler("/repo/ops-rehearsal-3-prep", collections="u/homer/w_2024_12/DM-43439")

In [3]:
vs = butler.get("visitSummary",  visit=7024040200866, instrument="LSSTComCamSim")

pixToArcseconds = np.mean([row.wcs.getPixelScale().asArcseconds() for row in vs]) 
SIGMA2FWHM = np.sqrt(8 * np.log(2))

e1 = (vs["psfIxx"] - vs["psfIyy"]) / (vs["psfIxx"] + vs["psfIyy"])
e2 = 2*vs["psfIxy"] / (vs["psfIxx"] + vs["psfIyy"])

outputDict = {
    # '5-sigma source count': len(calibrateRes.outputCat),
    'PSF FWHM': np.mean(vs["psfSigma"]) * SIGMA2FWHM * pixToArcseconds,
    'PSF e1': np.mean(e1),
    'PSF e2': np.mean(e2),
    'Sky mean': np.mean(vs["skyBg"]),
    'Sky RMS': np.mean(vs["skyNoise"]),
    'Variance plane mean': np.mean(vs["meanVar"]),
    'PSF star count': np.mean(vs["nPsfStar"]),
    'Astrometric bias': np.mean(vs["astromOffsetMean"]),
    'Astrometric scatter': np.mean(vs["astromOffsetStd"]),
    'Zeropoint': np.mean(vs["zeroPoint"]),
    "effTime": np.mean(vs["effTime"]), # these are new, I don't know if they're useful 
    "effTimePsfSigmaScale": np.mean(vs["effTimePsfSigmaScale"]), # these are new, I don't know if they're useful.
    "effTimeSkyBgScale": np.mean(vs["effTimeSkyBgScale"]), # these are new, I don't know if they're useful.
    "effTimeZeroPointScale": np.mean(vs["effTimeZeroPointScale"]), # these are new, I don't know if they're useful.
}

# flag all these as measured items to color the cell
labels = {"_" + k: "measured" for k in outputDict.keys()}
outputDict.update(labels)
print(outputDict)

{'PSF FWHM': 0.7175983664776634, 'PSF e1': 0.017793722, 'PSF e2': -0.019374542, 'Sky mean': 1772.0314, 'Sky RMS': 35.578793, 'Variance plane mean': 1177.3418, 'PSF star count': 1751.0, 'Astrometric bias': 0.009375281, 'Astrometric scatter': 0.004797545, 'Zeropoint': 31.10168, 'effTime': 18.510962, 'effTimePsfSigmaScale': 0.43105182, 'effTimeSkyBgScale': 0.016930519, 'effTimeZeroPointScale': 84.55661, '_PSF FWHM': 'measured', '_PSF e1': 'measured', '_PSF e2': 'measured', '_Sky mean': 'measured', '_Sky RMS': 'measured', '_Variance plane mean': 'measured', '_PSF star count': 'measured', '_Astrometric bias': 'measured', '_Astrometric scatter': 'measured', '_Zeropoint': 'measured', '_effTime': 'measured', '_effTimePsfSigmaScale': 'measured', '_effTimeSkyBgScale': 'measured', '_effTimeZeroPointScale': 'measured'}


In [4]:
# for the doc strings
vs.schema

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['I'](name="bbox_min_x", doc="bounding box (minimum)", units="pixel"), Key<I>(offset=8, nElements=1)),
    (Field['I'](name="bbox_min_y", doc="bounding box (minimum)", units="pixel"), Key<I>(offset=12, nElements=1)),
    (Field['I'](name="bbox_max_x", doc="bounding box (maximum)", units="pixel"), Key<I>(offset=16, nElements=1)),
    (Field['I'](name="bbox_max_y", doc="bounding box (maximum)", units="pixel"), Key<I>(offset=20, nElements=1)),
    (Field['L'](name="visit", doc="Visit number"), Key<L>(offset=24, nElements=1)),
    (Field['String'](name="physical_filter", doc="Physical filter", size=32), Key<String>(offset=32, nElements=32)),
    (Field['String'](name="band", doc="Name of band", size=32), Key<String>(offset=64, nElements=32)),
    (Field['F'](name="psfSigma", doc="PSF model second-moments determinant radius (center of chip) (pixel)", units="pixel"), Key<F>(offset=96, nElements=1))

In [7]:
# we also have:
vs.find(0).getVisitInfo()

VisitInfo(exposureTime=30, darkTime=30, date=2024-04-02T09:31:25.985098591, UT1=nan, ERA=5.8227 rad, boresightRaDec=(279.8870548486, -47.8262337699), boresightAzAlt=(nan, +nan), boresightAirmass=1.0764, boresightRotAngle=4.11588 rad, rotType=1, observatory=-30.2446N, -70.7494E  2663, weather=Weather(nan, nan, nan), instrumentLabel='LSSTComCamSim', id=7024040200866, focusZ=0, observationType='science', scienceProgram='12217', observationReason='survey', object='UNKNOWN', hasSimulatedContent=false)